In [35]:
# Import Numpy and Tensorflow.

import numpy as np
import tensorflow as tf


# Reproducibility.

def reset_graph(seed=1):
    np.random.seed(seed)
    tf.reset_default_graph()
    tf.set_random_seed(seed)


# Import California housing data from Scikit-Learn.

from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
housing_data_target = housing.target.reshape(-1, 1)
#housing_data_target = housing.target  # No need to reshape(-1, 1)


# Setup computational graph.

reset_graph ()

learning_rate = 0.01

#X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
#y = tf.constant(housing_data_target, dtype=tf.float32, name="y")

#theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
#y_pred = tf.matmul(X, theta , name="predictions")
#error = y_pred - y
#mse = tf.reduce_mean(tf.square(error), name="mse")
#optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#training_op = optimizer.minimize(mse)

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=9), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(mse)


# Execute using Mini-Batch Gradient Descent.

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = housing_data_plus_bias[indices]
    y_batch = housing_data_target[indices]
    return X_batch, y_batch

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
    
best_theta


array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan]], dtype=float32)

In [34]:
housing_data_target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])

In [ ]:



# Fetch batch function:

def fetch_batch(epoch, batch_index, batch_size):
    
    return X_batch, y_batch


# Set up computational graph:

import tensorflow as tf
reset_graph ()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing_data_target, dtype=tf.float32, name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf .matmul(X, theta , name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(mse)


# Execute:

init = tf.global_variables_initializer()

with
tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE=", mse.eval()) sess.run(training_op)
    best_theta = theta.eval()
